In [1]:
%pip install -q sentence-transformers

import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_distances
import hdbscan


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Geclustertes DataFrame laden
df = pd.read_excel("../../data/df_de_final_duplikate_markiert.xlsx", engine="openpyxl")


In [3]:
# Nur eine Frage pro Duplikat-Gruppe oder alle Einzel-Fragen
df_unique = df.drop_duplicates(subset=["Duplikat_Gruppe"], keep="first").copy()

print(f"Anzahl einzigartiger Fragen: {len(df_unique)}")


Anzahl einzigartiger Fragen: 1316


In [4]:
# Sentence-BERT Modell laden (multilingual, funktioniert gut auf Deutsch)
model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

# Fragetexte vorbereiten
texts = df_unique['Frage_Text'].tolist()

# Embeddings erzeugen
vectors = model.encode(texts, convert_to_numpy=True, show_progress_bar=True)


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

c:\Users\olive\git\EPRP\.conda\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\olive\.cache\huggingface\hub\models--sentence-transformers--distiluse-base-multilingual-cased-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

Batches:   0%|          | 0/42 [00:00<?, ?it/s]

In [5]:
# Cosine Distances berechnen (BERT-Embeddings sind normalerweise gut normalisiert)
distance_matrix = cosine_distances(vectors).astype(np.float64)


In [7]:
# HDBSCAN Clustering
clusterer = hdbscan.HDBSCAN(
    min_cluster_size=2,
    min_samples=1,
    metric="precomputed",
    cluster_selection_epsilon=0.1  # macht HDBSCAN etwas "weicher"
)
labels_hdbscan = clusterer.fit_predict(distance_matrix)

# Cluster-Labels in DataFrame speichern
df_unique["Cluster_HDBSCAN"] = labels_hdbscan


In [8]:
# Effektivität prüfen: Wie viele einzigartige Fragen wurden sinnvoll geclustert?
anzahl_einzigartige = len(df_unique)
anzahl_geclusterte_einzigartige = (df_unique["Cluster_HDBSCAN"] != -1).sum()
effektivitätsquote = anzahl_geclusterte_einzigartige / anzahl_einzigartige * 100

print(f"Anzahl einzigartiger Fragen: {anzahl_einzigartige}")
print(f"Anzahl durch HDBSCAN geclusterter einzigartiger Fragen: {anzahl_geclusterte_einzigartige}")
print(f"Effektivität des Clustering (in %): {effektivitätsquote:.2f}%")


Anzahl einzigartiger Fragen: 1316
Anzahl durch HDBSCAN geclusterter einzigartiger Fragen: 908
Effektivität des Clustering (in %): 69.00%


In [9]:
# Exportieren
df_unique.to_excel("../../data/df_de_final_clustered_bert.xlsx", index=False, engine="openpyxl")

print("Clustering mit BERT erfolgreich abgeschlossen und exportiert.")


Clustering mit BERT erfolgreich abgeschlossen und exportiert.


In [10]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import PatternFill

# 1. BERT-Cluster-Ergebnis laden
df = pd.read_excel("../../data/df_de_final_clustered_bert.xlsx", engine="openpyxl")

# 2. Nach Cluster sortieren
df_sorted = df.sort_values(by=["Cluster_HDBSCAN", "Frage_Text"]).reset_index(drop=True)

# 3. Neues Workbook erstellen
wb = Workbook()
ws = wb.active
ws.title = "BERT-Cluster sortiert"

# 4. Header schreiben
ws.append(df_sorted.columns.tolist())

# 5. Farben definieren
cluster_fill = PatternFill(start_color="D9E1F2", end_color="D9E1F2", fill_type="solid")  # hellblau für Cluster-Trennung

# 6. Cluster schreiben mit visueller Trennung
current_cluster = None

for idx, row in df_sorted.iterrows():
    cluster_id = row["Cluster_HDBSCAN"]
    
    # Wenn neues Cluster beginnt, füge eine Trennzeile ein
    if cluster_id != current_cluster:
        if current_cluster is not None:
            ws.append(["" for _ in range(len(df_sorted.columns))])  # Leere Zeile
            for cell in ws[ws.max_row]:
                cell.fill = cluster_fill
        
        current_cluster = cluster_id

    # Normale Datenzeile schreiben
    ws.append(row.tolist())

# 7. Speichern
output_path = "../../data/df_de_final_clustered_bert_sorted_with_separators.xlsx"
wb.save(output_path)

print(f"Datei erfolgreich gespeichert: {output_path}")


Datei erfolgreich gespeichert: ../../data/df_de_final_clustered_bert_sorted_with_separators.xlsx
